## Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from potentiel_solaire.constants import RESULTS_FOLDER

## Récupération des résultats des deux methodes

In [ ]:
results_with_simplified_method = gpd.read_file(RESULTS_FOLDER / "priotirized_schools_buildings.gpkg", layer="results_with_simplified_method")
roof_segments_with_mns = pd.read_csv(RESULTS_FOLDER / "roof_segments_with_mns.csv").rename(columns={"surface": "roof_surface"})

# Calculate total roof surface area for each building using the segmented roofs
results_with_segmented_roofs = roof_segments_with_mns.groupby("cleabs_bat").agg(
    roof_surface=("roof_surface", "sum"),
).reset_index()

# Merge the results from both methods
results_comparison_buildings = pd.merge(
    results_with_simplified_method[["cleabs_bat", "surface_totale_au_sol", "surface_utile"]],
    results_with_segmented_roofs[["cleabs_bat", "roof_surface"]],
    on="cleabs_bat",
)

## Comparaisons des résultats

In [ ]:
def plot_scatter(
    x: pd.Series,
    y: pd.Series,
    title: str, 
    xlabel: str,
    ylabel: str,
    xlim: tuple = None,
    ylim: tuple = None,
):
    """Plot the results of two series against each other.
    
    Args:
        x (pd.Series): The x-axis data.
        y (pd.Series): The y-axis data.
        title (str): The title of the plot.
        xlabel (str): The label for the x-axis.
        ylabel (str): The label for the y-axis.
        xlim (tuple, optional): The limits for the x-axis. Defaults to None.
        ylim (tuple, optional): The limits for the y-axis. Defaults to None.
    """
    plt.rcParams['figure.figsize'] = [20, 10]

    # plot identity line
    identity_line = [0, x.max()]
    plt.plot(identity_line, identity_line,"r--")

    # compare x and y
    plt.scatter(x, y, alpha=0.8)

    # Set the title and labels
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    #  Set the limits for x and y axes if provided
    if xlim:
        plt.xlim(xlim)
    
    if ylim:
        plt.ylim(ylim)

    # Show the plot 
    plt.show()

In [ ]:
plot_scatter(
    x=results_comparison_buildings["surface_utile"],
    y=results_comparison_buildings["roof_surface"],
    title="Surface utile (methode simplifiée) vs surface des toits (segmentation via MNS)",
    xlabel="Surface utile (m²)",
    ylabel="Surface des toits (m²)",
)

In [ ]:
results_comparison_buildings["euclidean_distance"] = abs(results_comparison_buildings["surface_utile"] - results_comparison_buildings["roof_surface"])
results_comparison_buildings["euclidean_distance"].describe()

## Verification si la surface au sol du batiment reste un facteur determinant pour la calcule de la surface des toits

In [ ]:
plot_scatter(
    x=results_comparison_buildings["surface_totale_au_sol"],
    y=results_comparison_buildings["roof_surface"],
    title="Surface au sol vs surface des toits (segmentation via MNS)",
    xlabel="Surface au sol (m²)",
    ylabel="Surface des toits (m²)",
)

## Verification de l'hypothèse des toits plats

In [ ]:
roof_segments_with_mns["flat_roof_surface"] = roof_segments_with_mns["roof_surface"].where(
    roof_segments_with_mns["slope_bin_min"] == 0, 0
)

total_flat_surface = roof_segments_with_mns["flat_roof_surface"].sum()
total_roof_surface = roof_segments_with_mns["roof_surface"].sum()
ratio_flat_roof = total_flat_surface / total_roof_surface

print(f"Surface des toits plats: {total_flat_surface} m²")
print(f"Surface totale des toits: {total_roof_surface} m²")
print(f"Ratio des toits plats: {ratio_flat_roof:.2%}")

roof_segments_with_mns["slope_bin_label"] = roof_segments_with_mns["slope_bin_min"].astype(str) + "-" + roof_segments_with_mns["slope_bin_max"].astype(str)
h = roof_segments_with_mns.groupby(by=["slope_bin_min", "slope_bin_label"])["roof_surface"].sum().reset_index().sort_values(by="slope_bin_min")

plt.rcParams['figure.figsize'] = [20, 10]
plt.bar(h["slope_bin_label"], h["roof_surface"])
plt.title("Surface des toits par pente")
plt.xlabel("Pente des toits (°)")
plt.ylabel("Surface des toits (m²)")
plt.xticks(rotation=45)

plt.show()

In [ ]:
agg_by_buildings = roof_segments_with_mns.groupby("cleabs_bat").agg(
    roof_surface=("roof_surface", "sum"),
    flat_roof_surface=("flat_roof_surface", "sum"),
).reset_index()


plot_scatter(
    x=agg_by_buildings["roof_surface"],
    y=agg_by_buildings["flat_roof_surface"],
    title="",
    xlabel="Surface des toits (m²)",
    ylabel="Surface plane des toits (m²)",
)